In [2]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
from bs4 import BeautifulSoup
import re
import json

# --크롬창을 숨기고 실행-- driver에 options를 추가해주면된다
# options = webdriver.ChromeOptions()
# options.add_argument('headless')

url = 'https://map.naver.com/v5/search'
driver = webdriver.Chrome('./chromedriver')  # 드라이버 경로
# driver = webdriver.Chrome('./chromedriver',chrome_options=options) # 크롬창 숨기기
driver.get(url)
key_word = '강선로 치킨'  # 검색어


# css 찾을때 까지 10초대기
def time_wait(num, code):
    try:
        wait = WebDriverWait(driver, num).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, code)))
    except:
        print(code, '태그를 찾지 못하였습니다.')
        driver.quit()
    return wait


# css를 찾을때 까지 10초 대기
time_wait(10, 'div.input_box > input.input_search')

# 검색창 찾기
search = driver.find_element_by_css_selector('div.input_box > input.input_search')
search.send_keys(key_word)  # 검색어 입력
search.send_keys(Keys.ENTER)  # 엔터버튼 누르기

res = driver.page_source  # 페이지 소스 가져오기
soup = BeautifulSoup(res, 'html.parser')  # html 파싱하여  가져온다

sleep(1)


# frame 변경 메소드
def switch_frame(frame):
    driver.switch_to.default_content()  # frame 초기화
    driver.switch_to.frame(frame)  # frame 변경
    res
    soup


# 페이지 다운
def page_down(num):
    body = driver.find_element_by_css_selector('body')
    body.click()
    for i in range(num):
        body.send_keys(Keys.PAGE_DOWN)


# frame 변경
switch_frame('searchIframe')
page_down(40)
sleep(5)

# 매장 리스트
store_list = driver.find_elements_by_css_selector('._22p-O') #driver.find_elements_by_css_selector('._1EKsQ')
# 페이지 리스트
next_btn = driver.find_elements_by_css_selector('._2ky45 > a') #동일

# dictionary 생성
store_dict = {'매장정보': []}
# 시작시간
start = time.time()
print('[크롤링 시작...]')

# 크롤링 (페이지 리스트 만큼)
for btn in range(len(next_btn))[1:]:  # next_btn[0] = 이전 페이지 버튼 무시 -> [1]부터 시작
    store_list
    for data in range(len(store_list)):  # 매장 리스트 만큼
        page = driver.find_elements_by_css_selector('._3Apve') #driver.find_elements_by_css_selector('.OXiLu')
        page[data].click()
        sleep(2)
        try:
            # 상세 페이지로 이동
            switch_frame('entryIframe')
            time_wait(5, '._3XamX')
            # 스크롤을 맨밑으로 1초간격으로 내린다.
            for down in range(3):
                sleep(1)
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # -----매장명 가져오기-----
            store_name = driver.find_element_by_css_selector('._3XamX').text

            # -----평점-----
            try:
                store_rating_list = driver.find_element_by_css_selector('._1A8_M').text
                store_rating = re.sub('별점', '', store_rating_list).replace('\n', '')  # 별점이라는 단어 제거
            except:
                pass
            print(store_rating)

            # -----주소(위치)-----
            try:
                store_addr_list = driver.find_elements_by_css_selector('._1aj6-')
                for i in store_addr_list:
                    store_addr = i.find_element_by_css_selector('._1h3B_').text
            except:
                pass
            print(store_addr)
            # -----전화번호 가져오기-----
            try:
                store_tel = driver.find_element_by_css_selector('._3ZA0S').text
            except:
                pass

            print(store_tel)
            # -----영업시간-----
            try:
                store_time_list = driver.find_elements_by_css_selector('._2vK84')  # 아니 태그가 그세 바뀌네ㅡ,.ㅡ
                for i in store_time_list:
                    store_time = i.find_element_by_css_selector('._3uEtO > time').text
            except:
                pass
            print(store_time)

            # -----썸네일 사진 주소-----
            try:
                thumb_list = driver.find_element_by_css_selector('.cb7hz') \
                    .value_of_css_property('background-image')  # css 속성명을 찾는다
                store_thumb = re.sub('url|"|\)|\(', '', thumb_list)  # url , (" ") 제거
            except:
                pass
            print(store_thumb)

            # ---- dict에 데이터 집어넣기----
            dict_temp = {
                'name': store_name,
                'tel': store_tel,
                'star': store_rating,
                'addr': store_addr,
                'time': store_time,
                'thumb': store_thumb
            }

            store_dict['매장정보'].append(dict_temp)

            print(f'{store_name} ...완료')
            switch_frame('searchIframe')
            sleep(1)

        except:
            print('ERROR!' * 3)

    # 다음 페이지 버튼
    if page[-1]:  # 마지막 매장일 경우 다음버튼 클릭
        next_btn[-1].click()
        sleep(2)
    else:
        print('페이지 인식 못함')
        break

print('[데이터 수집 완료]\n소요 시간 :', time.time() - start)
driver.quit()  # 작업이 끝나면 창을닫는다.

# json 파일로 저장
with open('data/store_data.json', 'w', encoding='utf-8') as f:
    json.dump(store_dict, f, indent=4, ensure_ascii=False)

C:\Users\admin\AppData\Local\Temp\ipykernel_5608\2673688434.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')  # 드라이버 경로
C:\Users\admin\AppData\Local\Temp\ipykernel_5608\2673688434.py:38: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  search = driver.find_element_by_css_selector('div.input_box > input.input_search')
C:\Users\admin\AppData\Local\Temp\ipykernel_5608\2673688434.py:58: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  body = driver.find_element_by_css_selector('body')
C:\Users\admin\AppData\Local\Temp\ipykernel_5608\2673688434.py:70: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  store_list = driver.find_elements_by

[크롤링 시작...]


C:\Users\admin\AppData\Local\Temp\ipykernel_5608\2673688434.py:97: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  store_name = driver.find_element_by_css_selector('._3XamX').text
C:\Users\admin\AppData\Local\Temp\ipykernel_5608\2673688434.py:101: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  store_rating_list = driver.find_element_by_css_selector('._1A8_M').text
C:\Users\admin\AppData\Local\Temp\ipykernel_5608\2673688434.py:109: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  store_addr_list = driver.find_elements_by_css_selector('._1aj6-')
C:\Users\admin\AppData\Local\Temp\ipykernel_5608\2673688434.py:117: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, va

4.47/5
경기 고양시 일산서구 강선로 140
경의중앙일산역 1번 출구에서950m
0507-0094-0571
새벽 00:00에 영업 종료
https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fvideo-phinf.pstatic.net%2F20211027_292%2F1635324599018SW6AA_JPEG%2Fd7912aa9-3702-11ec-917b-a0369ff9545c_03.jpg
페리카나 후곡점 ...완료
4.3/5
경기 고양시 일산서구 강선로 164 상가동 102호
경의중앙일산역 1번 출구에서670m
0507-1417-7760
23:30에 영업 종료
https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20170228_181%2F1488250011775NqjuK_PNG%2F186178517671100_0.png
60계 일산동점 ...완료
4.41/5
경기 고양시 일산서구 강선로 192 후곡11단지 상가 109호
경의중앙일산역 1번 출구에서408m
0507-1419-9969
23:00에 영업 종료
https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20210824_55%2F1629791213855Wks7m_PNG%2FGhpf-z3Q8dmP0M_Yp9f7nDp_.png
치킨마루 후곡11단지점 ...완료
4.41/5
경기 고양시 일산서구 강선로 187 후곡마을10단지분산상가 101호
경의중앙일산역 1번 출구에서507m
031-911-4955
23:00에 영업 종료
https://search.pstatic.net/common/?autoRotate=true&type=w56